# Proper Action

And shit!

In [40]:
from collections import deque
from math import floor

import six
import unittest

if six.PY3:
    from io import StringIO
else:
    from cStringIO import StringIO

# no uppercase, remove lookalikes (such a 1 and l)
SHORTENER_CHARS = 'abcdefghijkmnopqrstuvwxyz023456789'
SHORTENER_CHARS_COUNT = len(SHORTENER_CHARS)
SHORTENER_CHARS_MAP = { v: i for i, v in enumerate(SHORTENER_CHARS) }


class IPv4(object):
    """Representation of an IPv4 address."""
    
    @classmethod
    def expand(cls, value):
        """Expand a minified IPv4 address."""
        result = 0
    
        for index, letter in enumerate(value[::-1]):
            if SHORTENER_CHARS_MAP.get(letter, -1) < 0:
                raise Exception("Unsupported character '{}'".format(letter))

            # yields
            result = result * SHORTENER_CHARS_COUNT + SHORTENER_CHARS_MAP[letter]

        return IPv4(result)

    @classmethod
    def from_str(cls, value):
        """Create an IPv4 instance from a string IPv4 representation."""
        result = 0
        
        for octet in value.split('.'):
            result <<= 8
            result ^= int(octet)
    
        return IPv4(result)
    
    @classmethod
    def from_int(cls, value):
        """Create an IPv4 instance with an unsigned, 32-bit integer."""
        return IPv4(value)
    
    def __init__(self, value):
        """Construct an IPv4 address from an unsigned 32-bit integer."""
        self.value = value
    
    def as_str(self):
        """Format the IPv4 address as a octet-delimited string."""
        reducer = self.value
        octets = []
    
        for _ in range(4):
            octets.append(str(reducer & 0xFF))
            reducer >>= 8

        return '.'.join(octets)
    
    def as_int(self):
        """Return the underlying 32-bit integer."""
        return self.value
    
    def minify(self):
        """Minify the IP address to a string."""
        value = self.value
        
        if value == 0:
            return SHORTENER_CHARS[0]

        result = []

        while value > 0:
            value, rem = floor(value / SHORTENER_CHARS_COUNT), value % SHORTENER_CHARS_COUNT
            result.append(SHORTENER_CHARS[rem])

        return ''.join(result)


class Ipv4TestCase(unittest.TestCase):
    
    def test_from_str(self):
        self.assertEqual(0xa0a0a0a, IPv4.from_str('10.10.10.10').as_int())
        self.assertEqual(0x7f000001, IPv4.from_str('127.0.0.1').as_int())
        self.assertEqual(0, IPv4.from_str('0.0.0.0').as_int())
        
    def test_from_int(self):
        self.assertEqual(0xa0a0a0a, IPv4.from_int(0xa0a0a0a).as_int())
        self.assertEqual(0x7f000001, IPv4.from_int(0x7f000001).as_int())
        
    def test_as_str(self):
        self.assertEqual('10.10.10.10', IPv4.from_int(0xa0a0a0a).as_str())
        self.assertEqual('127.0.0.1', IPv4.from_int(0x7f000001).as_str())
        
    def test_minify(self):
        self.assertEqual('a', IPv4.from_int(0).minify())
        self.assertEqual('g2kbzd', IPv4.from_str('10.10.10.10').minify())
        self.assertEqual('jx7p6nb', IPv4.from_str('127.0.0.1').minify())
        
    def test_expand(self):
        self.assertEqual('a')
        self.assertEqual('10.10.10.10', IPv4.expand('g2kbzd').as_str())
        self.assertEqual('127.0.0.1', IPv4.expand('jx7p6nb').as_str())


_ = unittest.main(argv=[''], exit=False)

FE...
ERROR: test_expand (__main__.Ipv4TestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-40-c9b33bc54b11>", line 107, in test_expand
    self.assertEqual('a')
TypeError: assertEqual() missing 1 required positional argument: 'second'

FAIL: test_as_str (__main__.Ipv4TestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-40-c9b33bc54b11>", line 99, in test_as_str
    self.assertEqual('127.0.0.1', IPv4.from_int(0x7f000001).as_str())
AssertionError: '127.0.0.1' != '1.0.0.127'
- 127.0.0.1
?  --
+ 1.0.0.127
?        ++


----------------------------------------------------------------------
Ran 5 tests in 0.004s

FAILED (failures=1, errors=1)


# Ghetto Fast Implementation

Derived from [this answer in PHP](https://stackoverflow.com/a/9933990/128967).

In [4]:
# let's fucking roll
ALPH = 'abcdefghijk'
ALPH_MAP = { v: i for i, v in enumerate(ALPH) }
ALPH_LEN = len(ALPH)

def encode(val):
    if val < ALPH_LEN:
        return ALPH[val]
    
    result = []
    
    while val > 0:
        rem = int(val % ALPH_LEN)
        val = (val - rem) / ALPH_LEN
        result.append(ALPH[rem])
        
    return ''.join(result)
        

def decode(val):
    result = 0
    
    for index in range(len(val))[::-1]:
        letter = val[index]
        result += ALPH_MAP[letter] * pow(ALPH_LEN, index)

    return result

for v in range(0, ALPH_LEN * 10):
    encoded = encode(v)
    decoded = decode(encoded)
    # print("{} => {} => {} ({})".format(v, encoded, decoded, v == decoded))
    

In [43]:
def gen_ipv4_int(octet0, octet1, octet2, octet3):
    result = 0
    # octet 0
    result ^= octet0
    result <<= 8
    # octet 1
    result ^= octet1
    result <<= 8
    # octet 2
    result ^= octet2
    result <<= 8
    # octet 3
    result ^= octet3
    result <<= 8
    
    return result
    

def stringify_ipv4_int(value):
    octets = []
    
    for _ in range(4):
        octets.append(str(value & 0xFF))
        value >>= 8
        
    return '.'.join(octets)
    

localhost = gen_ipv4_int(127, 0, 0, 1)

print(hex(localhost))
print(stringify_ipv4_int(0x7f00000100))

0x7f00000100
0.1.0.0
